In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.4 MB/s eta 0:00:00


In [ ]:
from datasets import load_from_disk, load_dataset
from transformers import AutoTokenizer

In [ ]:
input_dataset_path = "/content/drive/MyDrive/Diff_Quality_Estimation"
max_seq_len = 1024
tokenizer_name = "unsloth/Qwen2.5-Coder-0.5B-bnb-4bit"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

In [ ]:
data_map = {
    'train': ['cls-train-chunk-0.jsonl', 'cls-train-chunk-3.jsonl', 'cls-train-chunk-1.jsonl', 'cls-train-chunk-2.jsonl'],
    'valid': ['cls-valid.jsonl'],
    'test': ['cls-test.jsonl']
}

ds = load_dataset(input_dataset_path, data_files=data_map)
ds

Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/23 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['oldf', 'patch', 'msg', 'id', 'y', 'idx', 'proj', 'lang'],
        num_rows: 265836
    })
    valid: Dataset({
        features: ['oldf', 'patch', 'msg', 'id', 'y', 'idx', 'proj', 'lang'],
        num_rows: 31252
    })
    test: Dataset({
        features: ['oldf', 'patch', 'msg', 'id', 'y', 'idx', 'proj', 'lang'],
        num_rows: 31252
    })
})

In [ ]:
my_prompt = """You are a code review expert skilled in all programming languages. You have been given a code snippet and the corresponding patch review request for it. Based on these 2 inputs, your task is to analyze whether the code needs a formal review from a human.
Return a 1 in the response if a review is needed, or a 0 if it is not. The response must contain only a single integer(0 or 1) and no text. Any deviation from this format will make the response invalid.
### Input:
{}
### Response: {}
"""
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    inputs       = [
        f"Original_Code:\n{old_code}\n\nPatch:\{patch}" for old_code, patch in zip(examples["oldf"], examples["patch"])
    ]
    outputs      = examples["y"]
    texts = []
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = my_prompt.format(input,output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts }

In [ ]:
def format_and_calculate_sizes(examples):
    inputs = []
    outputs = examples["y"]
    texts = []
    seq_lens = []
    for old_code, patch, output in zip(examples["oldf"], examples["patch"], outputs):
        input_text = f"Original_Code:\n{old_code}\n\nPatch:\{patch}"
        formatted_text = my_prompt.format(input_text, output) + EOS_TOKEN

        if len(formatted_text) > 50000:  # Skip massive entries
            texts.append("")
            seq_lens.append(0)
            continue
        texts.append(formatted_text)
        seq_lens.append(len(tokenizer(formatted_text)["input_ids"]))
    return {"text": texts, "seq_len": seq_lens}

In [ ]:
output_dataset_name = f"/content/drive/MyDrive/arpit_project/qwen_model/dataset_tokenized_1024_{tokenizer_name.replace('/','-')}/"
for split_name in ['test', 'train']:
    processed_ds = ds[split_name].map(format_and_calculate_sizes, batched=True, batch_size=1000)
    output_path = f"{output_dataset_name}/{split_name}"
    print(f"Saving {split_name} to {output_path}")
    processed_ds.save_to_disk(output_path)

Saving test to /content/drive/MyDrive/arpit_project/qwen_model/dataset_tokenized_1024_unsloth-Qwen2-1.5b-bnb-4bit//test


Saving the dataset (0/4 shards):   0%|          | 0/31252 [00:00<?, ? examples/s]

Map:   0%|          | 0/265836 [00:00<?, ? examples/s]

Saving train to /content/drive/MyDrive/arpit_project/qwen_model/dataset_tokenized_1024_unsloth-Qwen2-1.5b-bnb-4bit//train


Saving the dataset (0/30 shards):   0%|          | 0/265836 [00:00<?, ? examples/s]

In [ ]:
for split_name in ['valid', 'test', 'train']:
    split = load_from_disk(f"{output_dataset_name}/{split_name}")
    filtered = split.filter(lambda x: ( x['seq_len'] != 0 and x['seq_len'] < max_seq_len))
    output_path = f"/content/drive/MyDrive/arpit_project/qwen_model/dataset_filtered_{tokenizer_name}_{max_seq_len}/{split_name}"
    print(f"Saving filtered {split_name}")
    filtered.save_to_disk(output_path)

Filter:   0%|          | 0/31252 [00:00<?, ? examples/s]

Saving filtered valid


Saving the dataset (0/1 shards):   0%|          | 0/5956 [00:00<?, ? examples/s]

Filter:   0%|          | 0/31252 [00:00<?, ? examples/s]

Saving filtered test


Saving the dataset (0/1 shards):   0%|          | 0/6023 [00:00<?, ? examples/s]

Loading dataset from disk:   0%|          | 0/30 [00:00<?, ?it/s]

Filter:   0%|          | 0/265836 [00:00<?, ? examples/s]

Saving filtered train


Saving the dataset (0/5 shards):   0%|          | 0/36750 [00:00<?, ? examples/s]